# Task 1 Importing the dataset and deal with missing value with mean imputation

##### From the data, we know that missing value is represented by "?", so one of solution is declare it as na_value while the importing process using pandas

In [4]:
# 1. Importing dataframe using pandas and imputate the missing value
import pandas as pd
import numpy as np

class color:
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

# List to store DataFrames
invalid_values = ['?', " "]
df1 = pd.read_csv('/Users/mac/Documents/762/data/arrhythmia.csv', na_values = invalid_values)
df2 = pd.read_csv('/Users/mac/Documents/762/data/BCP.csv', na_values = invalid_values)
df3 = pd.read_csv('/Users/mac/Documents/762/data/website-phishing.csv', na_values = invalid_values)
dfs = [df1,df2,df3]

# Give Information File
df_name = ['arrhythmia', 'BCP', 'website-phishing']

for i in range(len(dfs)):
    print(df_name[i])
    print(dfs[i].head())

# Check whether there are missing/invalid values
print("Check Missing Values")
for i in range(len(dfs)):
    print(f"Checking for missing values in {df_name[i]}")
    missing = dfs[i].isnull().any()
    if missing.any():
        missing_columns = missing[missing].index.tolist()
        print(f"Missing values found in the following columns: {', '.join(missing_columns)}")
    else:
        print("No missing values found in this file")

print(color.BOLD+"Total Missing Value of Each File Before Imputation"+color.END)
print("Missing value in arrhythmia :",df1.isnull().sum().sum())
print("Missing value in BCP :",df2.isnull().sum().sum())
print("Missing value in website-phishing :",df3.isnull().sum().sum())

#Impute the invalid values only on df1 'arrhythmia'
for i in missing_columns:
    mean = df1[i].mean()
    df1[i].fillna(df1[i].mean(), inplace=True)
    
print(color.BOLD+"Total Missing Value of Each File After Imputation"+color.END)
print("Missing value in arrhythmia :",df1.isnull().sum().sum())
print("Missing value in BCP :",df2.isnull().sum().sum())
print("Missing value in website-phishing :",df3.isnull().sum().sum())

arrhythmia
   age   sex   height   weight   QRSduration   PRinterval   Q-Tinterval  \
0   75     0      190       80            91          193           371   
1   56     1      165       64            81          174           401   
2   54     0      172       95           138          163           386   
3   55     0      175       94           100          202           380   
4   75     0      190       80            88          181           360   

    Tinterval   Pinterval   QRS  ...  chV6_QwaveAmp  chV6_RwaveAmp  \
0         174         121   -16  ...            0.0            9.0   
1         149          39    25  ...            0.0            8.5   
2         185         102    96  ...            0.0            9.5   
3         179         143    28  ...            0.0           12.2   
4         177         103   -16  ...            0.0           13.1   

   chV6_SwaveAmp  chV6_RPwaveAmp  chV6_SPwaveAmp  chV6_PwaveAmp  \
0           -0.9             0.0             0.0  

# Task 2 Implement Decision Stump, Unpruned Decision tree and Pruned Decision tree

Constructing Object Oriented to build decision stump, unpruned decision tree and pruned decision tree to help me organized eah function and output.

In [4]:
import numpy as np
from collections import Counter

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split=1, max_depth=100, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)


    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # calculate the information gain
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold


    def _information_gain(self, y, X_column, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)

        # create children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        
        # calculate the weighted avg. entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        ''' function to compute entropy '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy

    def _most_common_label(self, y):
        def convert_to_hashable(item):
            if isinstance(item, np.ndarray):
                return tuple(convert_to_hashable(x) for x in item)
            return item

        if len(y) == 0:
            # If y is empty, return None or any other default value as appropriate
            return None

        converted_y = convert_to_hashable(y)
        counter = Counter(converted_y)

        if len(counter) == 0:
            # If Counter object is empty, return None or any other default value as appropriate
            return None

        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    def accuracy(self, y_test, y_pred):
        return np.sum(y_test == y_pred) / len(y_test)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_" + str(tree.feature), "<=", tree.threshold)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)


## Decision Stumps

### Arrythmia dataset

In [10]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

# Initialize an empty list to store accuracy values
acc_arrDSdf1 = []

# Iterate 10 times
for _ in range(10):
    data = df1
    X, y = df1.iloc[:, :-1].values, df1.iloc[:, -1].values.reshape(-1,1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=12345)

    clf = DecisionTree(max_depth=1)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    acc = clf.accuracy(y_test, predictions)
    
    # Append accuracy to acc_arr
    acc_arrDSdf1.append(acc)

    # Additional prints for debugging or observation
    print("Iteration:", _ + 1)    
# After all iterations, you can print or process the acc_arr as needed
print("Accuracy Array:", acc_arrDSdf1)
tree = clf.print_tree()
print("Tree:", tree)
print()

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Accuracy Array: [0.5217391304347826, 0.5217391304347826, 0.5217391304347826, 0.5217391304347826, 0.5217391304347826, 0.5217391304347826, 0.5217391304347826, 0.5217391304347826, 0.5217391304347826, 0.5217391304347826]
X_196 <= -0.7
 left:(1,)
 right:(2,)
Tree: None



### BCP Dataset

In [11]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming DecisionTree and its methods are already defined

# Initialize an empty list to store accuracy values
acc_arrDSdf2 = []

# Iterate 10 times
for _ in range(10):
    data = df2
    X, y = df2.iloc[:, :-1].values, df2.iloc[:, -1].values.reshape(-1,1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=12345)

    clf = DecisionTree(max_depth=1)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    acc = clf.accuracy(y_test, predictions)
    
    # Append accuracy to acc_arr
    acc_arrDSdf2.append(acc)

    # Additional prints for debugging or observation
    print("Iteration:", _ + 1)    
# After all iterations, you can print or process the acc_arr as needed
print("Accuracy Array:", acc_arrDSdf2)
tree = clf.print_tree()
print("Tree:", tree)
print()

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Accuracy Array: [0.9565217391304348, 0.9565217391304348, 0.9565217391304348, 0.9565217391304348, 0.9565217391304348, 0.9565217391304348, 0.9565217391304348, 0.9565217391304348, 0.9565217391304348, 0.9565217391304348]
X_2 <= 2
 left:(2,)
 right:(4,)
Tree: None



### Website-phising

In [12]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming DecisionTree and its methods are already defined

# Initialize an empty list to store accuracy values
acc_arrDSdf3 = []

# Iterate 10 times
for _ in range(10):
    data = df3
    X, y = df3.iloc[:, :-1].values, df3.iloc[:, -1].values.reshape(-1,1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=12345)

    clf = DecisionTree(max_depth=1)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    acc = clf.accuracy(y_test, predictions)
    
    # Append accuracy to acc_arr
    acc_arrDSdf3.append(acc)

    # Additional prints for debugging or observation
    print("Iteration:", _ + 1)    
# After all iterations, you can print or process the acc_arr as needed
print("Accuracy Array:", acc_arrDSdf3)
tree = clf.print_tree()
print("Tree:", tree)
print()

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Accuracy Array: [0.8905967450271248, 0.8905967450271248, 0.8905967450271248, 0.8905967450271248, 0.8905967450271248, 0.8905967450271248, 0.8905967450271248, 0.8905967450271248, 0.8905967450271248, 0.8905967450271248]
X_7 <= 0
 left:(-1,)
 right:(1,)
Tree: None



## Unpruned Decision Tree

### Arriythmia

In [13]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

# Initialize an empty list to store accuracy values
acc_arrUPTdf1 = []

# Iterate 10 times
for _ in range(10):
    data = df1
    X, y = df1.iloc[:, :-1].values, df1.iloc[:, -1].values.reshape(-1,1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=12345)

    clf = DecisionTree(max_depth=200)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    acc = clf.accuracy(y_test, predictions)
    
    # Append accuracy to acc_arr
    acc_arrUPTdf1.append(acc)

    # Additional prints for debugging or observation
    print("Iteration:", _ + 1)    
# After all iterations, you can print or process the acc_arr as needed
print("Accuracy Array:", acc_arrUPTdf1)
tree = clf.print_tree()
print("Tree:", tree)
print()


Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Accuracy Array: [0.7391304347826086, 0.6521739130434783, 0.717391304347826, 0.6739130434782609, 0.6739130434782609, 0.6956521739130435, 0.6739130434782609, 0.6956521739130435, 0.6956521739130435, 0.7391304347826086]
X_196 <= -0.7
 left:X_14 <= 58.0
  left:X_227 <= -11.9
    left:X_265 <= 0.5
        left:X_228 <= -42.3
                left:(16,)
                right:(6,)
        right:X_246 <= 4.3
                left:(6,)
                right:(1,)
    right:X_167 <= 9.1
        left:(10,)
        right:X_136 <= 48.0
                left:(16,)
                right:(6,)
  right:X_223 <= 0.5
    left:X_111 <= 20.0
        left:X_4 <= 87.0
                left:X_14 <= 94.0
                                left:X_75 <= 24.0
                                                                left:X_207 <= 18.0
                                                      

### BCP

In [14]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming DecisionTree and its methods are already defined

# Initialize an empty list to store accuracy values
acc_arrUPTdf2 = []

# Iterate 10 times
for _ in range(10):
    data = df2
    X, y = df2.iloc[:, :-1].values, df2.iloc[:, -1].values.reshape(-1,1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=12345)

    clf = DecisionTree(max_depth=200)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    acc = clf.accuracy(y_test, predictions)
    
    # Append accuracy to acc_arr
    acc_arrUPTdf2.append(acc)

    # Additional prints for debugging or observation
    print("Iteration:", _ + 1)    
# After all iterations, you can print or process the acc_arr as needed
print("Accuracy Array:", acc_arrUPTdf2)
tree = clf.print_tree()
print("Tree:", tree)
print()

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Accuracy Array: [0.9710144927536232, 0.9565217391304348, 0.9565217391304348, 0.9565217391304348, 0.9710144927536232, 0.9565217391304348, 0.9565217391304348, 0.9710144927536232, 0.9565217391304348, 0.9565217391304348]
X_2 <= 2
 left:X_8 <= 3
  left:X_1 <= 6
    left:X_6 <= 4
        left:(2,)
        right:X_5 <= 1
                left:(4,)
                right:(2,)
    right:X_6 <= 1
        left:(2,)
        right:(4,)
  right:X_3 <= 1
    left:X_0 <= 167528
        left:(2,)
        right:(4,)
    right:(4,)
 right:X_2 <= 4
  left:X_6 <= 3
    left:X_1 <= 4
        left:(2,)
        right:X_8 <= 1
                left:(2,)
                right:X_0 <= 1169049
                                left:(4,)
                                right:X_0 <= 1333877
                                                                left:(2,)
                             

### Website-phising

Due to indexing problem in web-pishing, i conduct some small tweak in my function for this datasets. But, i use some flow cart that used above

In [5]:
import numpy as np
from collections import Counter

class Nodedf3:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_leaf_nodedf3(self):
        return self.value is not None


class DecisionTreedf3:
    def __init__(self, min_samples_split=1, max_depth=100, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Nodedf3(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Nodedf3(best_feature, best_thresh, left, right)


    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # calculate the information gain
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold


    def _information_gain(self, y, X_column, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)

        # create children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        
        # calculate the weighted avg. entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy

    def _most_common_label(self, y):
        def convert_to_hashable(item):
            if isinstance(item, np.ndarray):
                return tuple(convert_to_hashable(x) for x in item)
            return item

        if len(y) == 0:
            # If y is empty, return None or any other default value as appropriate
            return None

        converted_y = convert_to_hashable(y)
        counter = Counter(converted_y)

        if len(counter) == 0:
            # If Counter object is empty, return None or any other default value as appropriate
            return None

        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, nodedf3):
        if nodedf3.is_leaf_nodedf3():
            return nodedf3.value

        if x[nodedf3.feature] <= x[nodedf3.threshold]:
            return self._traverse_tree(x, nodedf3.left)
        return self._traverse_tree(x, nodedf3.right)

    def accuracy(self, y_test, y_pred):
        return np.sum(y_test == y_pred) / len(y_test)
    
    def print_tree(self, tree=None, indent=" "):
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_" + str(tree.feature), "<=", tree.threshold)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    

In [16]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming DecisionTree and its methods are already defined

# Initialize an empty list to store accuracy values
acc_arrUPTdf3 = []

# Iterate 10 times
for _ in range(10):
    data = df3
    X, y = df3.iloc[:, :-1].values, df3.iloc[:, -1].values.reshape(-1,1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=12345)

    clf = DecisionTreedf3(max_depth=800)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    acc = clf.accuracy(y_test, predictions)
    
    # Append accuracy to acc_arr
    acc_arrUPTdf3.append(acc)

    # Additional prints for debugging or observation
    print("Iteration:", _ + 1)    
# After all iterations, you can print or process the acc_arr as needed
print("Accuracy Array:", acc_arrUPTdf3)


Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Accuracy Array: [0.46473779385171793, 0.46473779385171793, 0.46473779385171793, 0.46473779385171793, 0.46473779385171793, 0.46473779385171793, 0.46473779385171793, 0.46473779385171793, 0.46473779385171793, 0.46473779385171793]


#### For pruning process, i'm consider to adjust the maximum depth by retrieve the most optimum max_depth for each tree with finding the minimum approximate error (in this case i'm using validation error - training error). With assumtion I have dedicated 10 percent of my data just for cross validation, so validation error can act as unbiased estimator for test error, as i found from this article (https://stats.stackexchange.com/questions/480984/why-cross-validation-gives-biased-estimates-of-error)

### Pruned Tree

### Tuning the maximum depth Arrythmia

In [17]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

data = df1
X, y = df1.iloc[:, :-1].values, df1.iloc[:, -1].values.reshape(-1,1)
dp = [5,7,9,11,20]
ap = 1
opt_d = None

for d in dp:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=12345)
    clf = DecisionTree(max_depth=d)

    print("Case of Max depth:", d)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    y_pred = clf.predict(X_train)

    # Print the accuracy
    acc_test = clf.accuracy(y_test, predictions)
    print("Accuracy Test :", acc_test)
    acc_train = clf.accuracy(y_train, y_pred)
    print("Accuracy Training :", acc_train)
    val_err = 1 - acc_test
    train_err = 1 - acc_train
    print("Validation Error :", val_err)
    print("Training Error :", train_err)
    approx_err = abs(val_err - train_err)
    if approx_err < ap:
        ap = approx_err
        opt_d = d
    
print("Optimal Pruning depth:", opt_d)

Case of Max depth: 5
Accuracy Test : 0.7608695652173914
Accuracy Training : 0.8103448275862069
Validation Error : 0.23913043478260865
Training Error : 0.18965517241379315
Case of Max depth: 7
Accuracy Test : 0.6739130434782609
Accuracy Training : 0.9088669950738916
Validation Error : 0.32608695652173914
Training Error : 0.09113300492610843
Case of Max depth: 9
Accuracy Test : 0.8043478260869565
Accuracy Training : 0.9704433497536946
Validation Error : 0.19565217391304346
Training Error : 0.029556650246305383
Case of Max depth: 11
Accuracy Test : 0.6521739130434783
Accuracy Training : 0.9975369458128078
Validation Error : 0.34782608695652173
Training Error : 0.0024630541871921707
Case of Max depth: 20
Accuracy Test : 0.782608695652174
Accuracy Training : 1.0
Validation Error : 0.21739130434782605
Training Error : 0.0
Optimal Pruning depth: 5


### Tuning the maximum depth BCP

In [18]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

data = df2
X, y = df2.iloc[:, :-1].values, df2.iloc[:, -1].values.reshape(-1,1)
dp = [5,7,11,20]
ap = 1
opt_d = None
#_node = Node(feature=7, threshold=0)
for d in dp:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=12345)
    clf = DecisionTree(max_depth=d)

    print("Case of Max depth:", d)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    y_pred = clf.predict(X_train)

    # Print the accuracy
    acc_test = clf.accuracy(y_test, predictions)
    print("Accuracy Test :", acc_test)
    acc_train = clf.accuracy(y_train, y_pred)
    print("Accuracy Training :", acc_train)
    val_err = 1 - acc_test
    train_err = 1 - acc_train
    print("Validation Error :", val_err)
    print("Training Error :", train_err)
    approx_err = abs(val_err - train_err)
    if approx_err < ap:
        ap = approx_err
        opt_d = d
    
print("Optimal Pruning depth:", opt_d)

Case of Max depth: 5
Accuracy Test : 0.9565217391304348
Accuracy Training : 0.9804560260586319
Validation Error : 0.04347826086956519
Training Error : 0.019543973941368087
Case of Max depth: 7
Accuracy Test : 0.9710144927536232
Accuracy Training : 0.995114006514658
Validation Error : 0.02898550724637683
Training Error : 0.004885993485342022
Case of Max depth: 11
Accuracy Test : 0.9565217391304348
Accuracy Training : 1.0
Validation Error : 0.04347826086956519
Training Error : 0.0
Case of Max depth: 20
Accuracy Test : 0.9710144927536232
Accuracy Training : 1.0
Validation Error : 0.02898550724637683
Training Error : 0.0
Optimal Pruning depth: 5


### Tuning the maximum depth Website_phising

In [19]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

data = df3
X, y = df3.iloc[:, :-1].values, df3.iloc[:, -1].values.reshape(-1,1)
dp = [30,40,50,70,90]
ap = 1
opt_d = None
for d in dp:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=12345)
    clf = DecisionTreedf3(max_depth=d)

    print("Case of Max depth:", d)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    y_pred = clf.predict(X_train)

    # Print the accuracy
    acc_test = clf.accuracy(y_test, predictions)
    print("Accuracy Test :", acc_test)
    acc_train = clf.accuracy(y_train, y_pred)
    print("Accuracy Training :", acc_train)
    val_err = 1 - acc_test
    train_err = 1 - acc_train
    print("Validation Error :", val_err)
    print("Training Error :", train_err)
    approx_err = abs(val_err - train_err)
    if approx_err < ap:
        ap = approx_err
        opt_d = d
    
print("Optimal Pruning depth:", opt_d)

Case of Max depth: 30
Accuracy Test : 0.46473779385171793
Accuracy Training : 0.4678862197205749
Validation Error : 0.5352622061482821
Training Error : 0.5321137802794251
Case of Max depth: 40
Accuracy Test : 0.5750452079566004
Accuracy Training : 0.5546286058900392
Validation Error : 0.4249547920433996
Training Error : 0.44537139410996085
Case of Max depth: 50
Accuracy Test : 0.46473779385171793
Accuracy Training : 0.4678862197205749
Validation Error : 0.5352622061482821
Training Error : 0.5321137802794251
Case of Max depth: 70
Accuracy Test : 0.46473779385171793
Accuracy Training : 0.4678862197205749
Validation Error : 0.5352622061482821
Training Error : 0.5321137802794251
Case of Max depth: 90
Accuracy Test : 0.46473779385171793
Accuracy Training : 0.4685898080209066
Validation Error : 0.5352622061482821
Training Error : 0.5314101919790934
Optimal Pruning depth: 30


### Arrythmia Pruned Tree

In [20]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

# Initialize an empty list to store accuracy values
acc_arrPTdf1 = []

# Iterate 10 times
for _ in range(10):
    data = df1
    X, y = df1.iloc[:, :-1].values, df1.iloc[:, -1].values.reshape(-1,1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=12345)

    clf = DecisionTree(max_depth=5)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    acc = clf.accuracy(y_test, predictions)
    
    # Append accuracy to acc_arr
    acc_arrPTdf1.append(acc)

    # Additional prints for debugging or observation
    print("Iteration:", _ + 1)    
# After all iterations, you can print or process the acc_arr as needed
print("Accuracy Array:", acc_arrPTdf1)
tree = clf.print_tree()
print("Tree:", tree)
print()

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Accuracy Array: [0.717391304347826, 0.7391304347826086, 0.7608695652173914, 0.7391304347826086, 0.8043478260869565, 0.7391304347826086, 0.7391304347826086, 0.7391304347826086, 0.7391304347826086, 0.7391304347826086]
X_196 <= -0.7
 left:X_14 <= 58.0
  left:X_227 <= -11.9
    left:X_199 <= -0.4
        left:X_189 <= 0.0
                left:(1,)
                right:(6,)
        right:X_161 <= 2.9
                left:(16,)
                right:(6,)
    right:X_167 <= 9.1
        left:(10,)
        right:X_6 <= 408.0
                left:(6,)
                right:(16,)
  right:X_223 <= 0.5
    left:X_111 <= 20.0
        left:X_4 <= 87.0
                left:(1,)
                right:(1,)
        right:X_68 <= 28.0
                left:(15,)
                right:(3,)
    right:X_245 <= 1.1
        left:X_225 <= -0.7
                left:(10,)
            

## BCP Pruned Tree

In [21]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

# Initialize an empty list to store accuracy values
acc_arrPTdf2 = []

# Iterate 10 times
for _ in range(10):
    data = df2
    X, y = df2.iloc[:, :-1].values, df2.iloc[:, -1].values.reshape(-1,1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=12345)

    clf = DecisionTree(max_depth=5)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    acc = clf.accuracy(y_test, predictions)
    
    # Append accuracy to acc_arr
    acc_arrPTdf2.append(acc)

    # Additional prints for debugging or observation
    print("Iteration:", _ + 1)    
# After all iterations, you can print or process the acc_arr as needed
print("Accuracy Array:", acc_arrPTdf2)
tree = clf.print_tree()
print("Tree:", tree)
print()

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Accuracy Array: [0.9565217391304348, 0.9565217391304348, 0.9710144927536232, 0.9710144927536232, 0.9565217391304348, 0.9565217391304348, 0.9710144927536232, 0.9565217391304348, 0.9710144927536232, 0.9565217391304348]
X_2 <= 2
 left:X_8 <= 3
  left:X_1 <= 6
    left:X_6 <= 4
        left:(2,)
        right:X_5 <= 1
                left:(4,)
                right:(2,)
    right:X_6 <= 1
        left:(2,)
        right:(4,)
  right:X_1 <= 4
    left:X_0 <= 167528
        left:(2,)
        right:(4,)
    right:(4,)
 right:X_2 <= 4
  left:X_6 <= 3
    left:X_1 <= 4
        left:(2,)
        right:X_8 <= 1
                left:(2,)
                right:(4,)
    right:X_3 <= 4
        left:X_4 <= 5
                left:(4,)
                right:(4,)
        right:(4,)
  right:X_7 <= 4
    left:X_1 <= 6
        left:X_6 <= 8
                left:(4,)
            

### Website_phising Pruned Tree

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

# Initialize an empty list to store accuracy values
acc_arrPTdf3 = []

# Iterate 10 times
#for _ in range(1):
data = df3
X, y = df3.iloc[:, :-1].values, df3.iloc[:, -1].values.reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=12345)
clf = DecisionTreedf3(max_depth=30)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
acc = clf.accuracy(y_test, predictions)
    
    # Append accuracy to acc_arr
#acc_arrPTdf3.append(acc)

    # Additional prints for debugging or observation 
# After all iterations, you can print or process the acc_arr as needed
print("Accuracy Array:", acc_arrPTdf3)
tree = clf.print_tree()

# Task 3 Select Your Hyperparameters such as Depth

### As the explanation of tuning decison tree in task 2, I have used approximated error that I estimated from cross validation error (as unbiased estimator of test error) and training error to find the most optimum number of maximum depth. I am choosing the most minimum approximated error between training and validation as the most balnced trade-off between push down the training error and test error.

# Task 4 Compare the three methods used in task 2 and determine if any are performing significantly worse on each dataset

#### After running, we have 3 methods for each 3 dataset. Because i'm just running 10 times for each model and each dataset, I am decided to test the performance (in this term I use accuracy) between 3 methods used for each dataset

## Arrythmia

In [1]:
from scipy.stats import kruskal

stumps = [0.5217391304347826, 0.5217391304347826, 0.5217391304347826, 0.5217391304347826, 0.5217391304347826, 0.5217391304347826, 0.5217391304347826, 0.5217391304347826, 0.5217391304347826, 0.5217391304347826]
unpruned = [0.7391304347826086, 0.6521739130434783, 0.717391304347826, 0.6739130434782609, 0.6739130434782609, 0.6956521739130435, 0.6739130434782609, 0.6956521739130435, 0.6956521739130435, 0.7391304347826086]
pruned = [0.717391304347826, 0.7391304347826086, 0.7608695652173914, 0.7391304347826086, 0.8043478260869565, 0.7391304347826086, 0.7391304347826086, 0.7391304347826086, 0.7391304347826086, 0.7391304347826086]

# Kruskal-Wallis Test
statistic, p_value = kruskal(stumps, unpruned, pruned)

# Test Result
print("Statistic:", statistic)
print("p-value:", p_value)

# Hypothesys
alpha = 0.05
if p_value < alpha:
    print("Significance difference.")
else:
    print("Insignificance difference.")


Statistic: 25.238490835515346
p-value: 3.307739879181062e-06
Significance difference.


There is significance evidence to claim that 3 methods performing significantly worse on Arrythmia dataset

## BCP

In [2]:
from scipy.stats import kruskal

stumps = [0.9565217391304348, 0.9565217391304348, 0.9565217391304348, 0.9565217391304348, 0.9565217391304348, 0.9565217391304348, 0.9565217391304348, 0.9565217391304348, 0.9565217391304348, 0.9565217391304348]
unpruned = [0.9710144927536232, 0.9565217391304348, 0.9565217391304348, 0.9565217391304348, 0.9710144927536232, 0.9565217391304348, 0.9565217391304348, 0.9710144927536232, 0.9565217391304348, 0.9565217391304348]
pruned = [0.9565217391304348, 0.9565217391304348, 0.9710144927536232, 0.9710144927536232, 0.9565217391304348, 0.9565217391304348, 0.9710144927536232, 0.9565217391304348, 0.9710144927536232, 0.9565217391304348]

# Kruskal-Wallis Test
statistic, p_value = kruskal(stumps, unpruned, pruned)

# Test Result
print("Statistic:", statistic)
print("p-value:", p_value)

# Hypothesys
alpha = 0.05
if p_value < alpha:
    print("Significance difference.")
else:
    print("Insignificance difference.")

Statistic: 4.683229813664595
p-value: 0.09617220360937143
Insignificance difference.


There no evidence to claim that 3 methods performing significantly worse on BCP dataset

## Website-Phising

In [3]:
from scipy.stats import kruskal

stumps = [0.8905967450271248, 0.8905967450271248, 0.8905967450271248, 0.8905967450271248, 0.8905967450271248, 0.8905967450271248, 0.8905967450271248, 0.8905967450271248, 0.8905967450271248, 0.8905967450271248]
unpruned = [0.46473779385171793, 0.46473779385171793, 0.46473779385171793, 0.46473779385171793, 0.46473779385171793, 0.46473779385171793, 0.46473779385171793, 0.46473779385171793, 0.46473779385171793, 0.46473779385171793]

# Kruskal-Wallis Test
statistic, p_value = kruskal(stumps, unpruned)

# Test Result
print("Statistic:", statistic)
print("p-value:", p_value)

# Hypothesys
alpha = 0.05
if p_value < alpha:
    print("Significance difference.")
else:
    print("Insignificance difference.")

Statistic: 18.99999999999999
p-value: 1.3071845366763076e-05
Significance difference.


For the website, I just compare the two methods because i don't know, before today my function works but not for today in injury time and i couldn't fix it due to data rate limit error. When i am trying to set data rate limit through my terminal, it is not works. 
So we can concude that there is significance evidence to said that unpruned tree performing significantly worse on Website dataset